In [1]:
# !pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [15]:
# Installing Dependencies

In [5]:
!pip install huggingface_hub datasets

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [9]:
from datasets import load_dataset

dataset = load_dataset("aegean-ai/sidewalks", split="train")
print(dataset)

Dataset({
    features: ['filename', 'tfw', 'tif', 'label_tif', 'label_tfw'],
    num_rows: 180000
})


In [10]:
!pip install fiftyone
!pip install torch torchvision torchaudio
!pip install pyyaml scipy

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.0/943.0 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 12.5 MB/s eta 0:00

In [12]:
#  Loading the Sidewalk Dataset into FiftyOne

In [13]:
import fiftyone as fo
import os
import PIL.Image
import io
import numpy as np

In [88]:
# Load dataset from Hugging Face
dataset = load_dataset("aegean-ai/sidewalks", split="train")

# Define dataset name
dataset_name = "sidewalks_dataset_with_masks"

# Check if the dataset exists, if so, delete it
if dataset_name in fo.list_datasets():
    fo.delete_dataset(dataset_name)

# Create a new FiftyOne dataset
fo_dataset = fo.Dataset(name=dataset_name)

# Convert dataset into FiftyOne format (limit to 20 images)
samples = []
for i in range(20):
    sample_dict = dataset[i]

    # Load image and mask
    img = sample_dict["tif"]
    mask = sample_dict["label_tif"]

    # Convert mask to grayscale
    mask = mask.convert("L")

    # Save image & mask
    img_path = f"/content/temp_image_{i}.png"
    mask_path = f"/content/temp_mask_{i}.png"
    img.save(img_path)
    mask.save(mask_path)

    # Create a FiftyOne sample
    sample = fo.Sample(filepath=img_path)
    sample["ground_truth"] = fo.Segmentation(mask_path=mask_path)

    samples.append(sample)

# Add samples to the FiftyOne dataset
fo_dataset.add_samples(samples)

# Launch FiftyOne
session = fo.launch_app()
session.dataset = fo_dataset


In [15]:
# Running Detectron2 Inference on Sidewalk Dataset

In [16]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo

# Configure Mask R-CNN with pre-trained weights
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

# Create the predictor
predictor = DefaultPredictor(cfg)
print("Detectron2 model loaded successfully!")

model_final_f10217.pkl: 178MB [00:00, 200MB/s]                           


Detectron2 model loaded successfully!


In [17]:
#  Saving Detectron2 Predictions in FiftyOne

In [87]:
import cv2

# Load your FiftyOne dataset
dataset = fo.load_dataset("sidewalks_dataset_with_masks")

# Run inference on each image in the dataset
for sample in dataset.iter_samples():
    image = cv2.imread(sample.filepath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Run Detectron2 inference
    outputs = predictor(image)

    # Extract predicted segmentation masks
    instances = outputs["instances"].to("cpu")
    masks = instances.pred_masks.numpy().astype(np.uint8) * 255

    # Save the first predicted mask (if available)
    if len(masks) > 0:
        mask_path = sample.filepath.replace(".png", "_predicted_mask.png")
        mask_img = PIL.Image.fromarray(masks[0])
        mask_img.save(mask_path)

        # Attach predicted mask to FiftyOne
        sample["predictions"] = fo.Segmentation(mask_path=mask_path)
        sample.save()

print("Predictions saved to FiftyOne dataset!")

session = fo.launch_app()
session.dataset = dataset

In [85]:
# The features of the model: 25 points

In [ ]:
############## Model: Detectron2’s Mask R-CNN ##############
# Framework: Detectron2 (by Facebook Research).
# Base Model: Mask R-CNN (ResNet-50 with Feature Pyramid Network)
# Config file: "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
# Pre-trained on COCO dataset.
# Uses Region of Interest (ROI) Heads for instance segmentation.
# Configuration Settings:
# Confidence threshold set to 0.5 for predictions.
# Pre-trained weights are loaded from the Detectron2 Model Zoo.

